# Importing the data and installing the required libraries:

In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import dates
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np

In [25]:
calender=pd.read_csv('calendar.csv',parse_dates=True)
sales=pd.read_csv('sales_train_validation.csv',parse_dates=True)
prices=pd.read_csv('sell_prices.csv')
calender['date']=pd.to_datetime(calender['date'])

In [32]:
#Converting dtypes to 'categorical' and filling nans with -1( we use -1 to further reduce the dataset size as compared to nans) for all the 3 dataframes:
calender=calender.fillna(-1)
calender[['event_type_1','event_type_2','event_name_1','event_name_2']]=calender[['event_type_1','event_type_2','event_name_1','event_name_2']].astype(('category'))

sales[['id','item_id','cat_id','store_id','state_id']]=sales[['id','item_id','cat_id','store_id','state_id']].astype('category')

# For prices column we combine the item_id and store_id to form the id of the data which can later be joined with sales dataframe:
prices['id']=prices['item_id']+'_'+prices['store_id']+'_evaluation'

prices[['id','store_id','item_id']]=prices[['id','store_id','item_id']].astype('category')
# We also drop store_id and item_id as they no longer play any role in the dataset and all the information is stored in 'id'.
prices.drop(['store_id','item_id'],axis=1,inplace=True)
# We also drop dept_id from sales as we will note be using the column:
sales.drop('dept_id',axis=1,inplace=True)

In [33]:
# This very convinient piece of code is commonly found on kaggle competitions which performs the above tasks for all the rows:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    

In [34]:
# Applying the above function to the prices dataframe and calander dataframe:
# We apply the function to sales dataframe after applying melt to sales:
prices=reduce_mem_usage(prices)
calender=reduce_mem_usage(calender)

Mem. usage decreased to 40.63 Mb (65.8% reduction)
Mem. usage decreased to  0.07 Mb (55.2% reduction)


In [36]:
# Next, we make the dataframe sales into a more usable format where each 
# day is a column rather than a row. This essentially makes the table vertical.

# We use pd.melt to do the task above, which essentially bring the table to the format given below:
sales=pd.melt(sales,id_vars=['id','item_id','cat_id','store_id','state_id'])
sales.head()

,id,item_id,cat_id,store_id,state_id,variable,value
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES,CA_1,CA,d_1,0


In [70]:
prices.to_csv('prices.csv')

In [71]:
calender.to_csv('calender.csv')

In [73]:
sales=reduce_mem_usage(sales)

Mem. usage decreased to 947.22 Mb (0.0% reduction)


In [74]:
sales.to_csv('sales.csv')

In [68]:
# Here we merge all three dataframes:
temp=pd.merge(calender[['date','d','wm_yr_wk','event_name_1','event_type_1', 
                        'event_name_2','event_type_2','snap_CA', 'snap_TX', 'snap_WI']],
              sales,left_on='d',right_on='variable',how='inner')
temp

,date,d,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,id,item_id,cat_id,store_id,state_id,variable,value
0,2011-01-29,d_1,11101,-1,-1,-1,-1,0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES,CA_1,CA,d_1,0
1,2011-01-29,d_1,11101,-1,-1,-1,-1,0,0,0,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES,CA_1,CA,d_1,0
2,2011-01-29,d_1,11101,-1,-1,-1,-1,0,0,0,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES,CA_1,CA,d_1,0
3,2011-01-29,d_1,11101,-1,-1,-1,-1,0,0,0,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES,CA_1,CA,d_1,0
4,2011-01-29,d_1,11101,-1,-1,-1,-1,0,0,0,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,2016-04-24,d_1913,11613,-1,-1,-1,-1,0,0,0,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS,WI_3,WI,d_1913,1
58327366,2016-04-24,d_1913,11613,-1,-1,-1,-1,0,0,0,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS,WI_3,WI,d_1913,0
58327367,2016-04-24,d_1913,11613,-1,-1,-1,-1,0,0,0,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS,WI_3,WI,d_1913,0
58327368,2016-04-24,d_1913,11613,-1,-1,-1,-1,0,0,0,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS,WI_3,WI,d_1913,3


In [ ]:
temp.to_csv('temp.csv')

In [69]:
df=pd.merge(temp,prices,left_on=['id','wm_yr_wk'],
            right_on=['id','wm_yr_wk'],how='inner',
            validate='m:1')
df

,date,d,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,id,item_id,cat_id,store_id,state_id,variable,value,sell_price


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 18 columns):
date            0 non-null datetime64[ns]
d               0 non-null object
wm_yr_wk        0 non-null int16
event_name_1    0 non-null category
event_type_1    0 non-null category
event_name_2    0 non-null category
event_type_2    0 non-null category
snap_CA         0 non-null int8
snap_TX         0 non-null int8
snap_WI         0 non-null int8
id              0 non-null object
item_id         0 non-null category
cat_id          0 non-null category
store_id        0 non-null category
state_id        0 non-null category
variable        0 non-null object
value           0 non-null int16
sell_price      0 non-null float16
dtypes: category(8), datetime64[ns](1), float16(1), int16(2), int8(3), object(3)
memory usage: 106.4+ KB


In [ ]:
# We get rid of the columns on which the dataframe was joined on as we already have the date column instead:
df.drop(['d','variable','wm_yr_wk'],axis=1,inplace=True)
# Rearranging the columns to our convinience:
cols=['date','id', 'item_id', 'cat_id', 'store_id', 'state_id','sell_price','event_name_1', 'event_type_1', 'event_name_2','event_type_2', 'snap_CA', 'snap_TX', 'snap_WI','value']
df=df[cols]

In [39]:
df

,date,id,item_id,cat_id,store_id,state_id,sell_price,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,value


In [15]:
df.tail()

,date,id,item_id,cat_id,store_id,state_id,sell_price,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,value


In [16]:
# Given below is information on the various columns of the dataframe:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 15 columns):
date            0 non-null datetime64[ns]
id              0 non-null object
item_id         0 non-null category
cat_id          0 non-null category
store_id        0 non-null category
state_id        0 non-null category
sell_price      0 non-null float16
event_name_1    0 non-null category
event_type_1    0 non-null category
event_name_2    0 non-null category
event_type_2    0 non-null category
snap_CA         0 non-null int8
snap_TX         0 non-null int8
snap_WI         0 non-null int8
value           0 non-null int16
dtypes: category(8), datetime64[ns](1), float16(1), int16(1), int8(3), object(1)
memory usage: 106.4+ KB


In [17]:
df.tail()

,date,id,item_id,cat_id,store_id,state_id,sell_price,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,value


In [18]:
df[['date','value']].groupby('date').agg({'value':'sum'}).plot(figsize=(20,8),grid=True);

TypeError: no numeric data to plot

In [ ]:
df.loc[(df.date>'2015-01-01')&(df.date<'2016-01-01')][['date','value']].groupby('date').agg({'value':'sum'}).plot(figsize=(20,8),grid=True);

In [ ]:
ax=df.loc[(df.date>'2014-08-01')&(df.date<'2014-09-01')][['date','value']].groupby('date').agg({'value':'sum'}).plot(figsize=(20,8))
ax.xaxis.set_minor_locator(dates.DayLocator())
ax.xaxis.set_minor_formatter(dates.DateFormatter("%a-%B-%d"))
ax.tick_params(which='minor', rotation=45)
ax.grid(b=True, which='minor')

In [ ]:
storewise=df[['date','store_id','value']].groupby(['date','store_id']).agg({'value':'sum'})
storewise.reset_index(inplace=True)
storewise.pivot(index="date", columns="store_id", values="value").rolling(window=90).mean().plot(figsize=(20,8),grid=True,title='Sum of sales by store');

In [ ]:
category_wise=df[['date','cat_id','value']].groupby(['date','cat_id']).agg({'value':'sum'})
category_wise.reset_index(inplace=True)
category_wise.pivot(index="date", columns="cat_id", values="value").rolling(window=90).mean().plot(figsize=(20,8),grid=True,title='Sum of sales by category');

In [ ]:
statewise=df[['date','state_id','value']].groupby(['date','state_id']).agg({'value':'sum'})
statewise.reset_index(inplace=True)
statewise.pivot(index="date", columns="state_id", values="value").rolling(window=90).mean().plot(figsize=(20,8),grid=True,title='Sum of sales by state');

In [ ]:
item1=df[['item_id','sell_price','value']].loc[df['item_id']=='HOBBIES_1_008']
sns.barplot(x='sell_price',y='value',data=item1).set_title('Item1')
sns.set(rc={'figure.figsize':(10,5)})
plt.show()

In [ ]:
item1= df[['cat_id','sell_price','value']].loc[df['cat_id']=='FOODS']
sns.scatterplot(x='sell_price',y='value',data=item1).set_title('Effect of price on sales for food')
sns.set(rc={'figure.figsize':(10,5)})
plt.show()

In [ ]:
ax=sns.barplot(x='event_name_1',y='value',data=df[['event_name_1','value']].groupby('event_name_1').agg({'value':'mean'}).sort_values(['value']).reset_index())
ax.tick_params(which='both',rotation=90)
sns.set(rc={'figure.figsize':(20,8)})

In [ ]:
fig, ax =plt.subplots(1,3)
sns.barplot(x='snap_CA',y='value',data=df[['snap_CA','value']].groupby('snap_CA').agg({'value':'mean'}).sort_values(['value']).reset_index(),ax=ax[0])
sns.set(rc={'figure.figsize':(10,6)})
sns.barplot(x='snap_TX',y='value',data=df[['snap_TX','value']].groupby('snap_TX').agg({'value':'mean'}).sort_values(['value']).reset_index(),ax=ax[1])
sns.barplot(x='snap_WI',y='value',data=df[['snap_WI','value']].groupby('snap_WI').agg({'value':'mean'}).sort_values(['value']).reset_index(),ax=ax[2])
plt.show()